<a href="https://colab.research.google.com/github/bintangnabiil/Deep_Learning/blob/main/Deep_Learning_Week1_Infrared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Load dataset langsung dari file yang telah dimasukkan
file_name = "Infrared.csv"
df = pd.read_csv(file_name)

# Mengisi missing values
if 'Distance' in df.columns:
    df.loc[:, 'Distance'] = df['Distance'].fillna(df['Distance'].median())

# One-Hot Encoding untuk variabel kategori
categorical_cols = ['Gender', 'Age', 'Ethnicity']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cats = encoder.fit_transform(df[categorical_cols])

df_encoded = pd.concat(
    [df.drop(columns=categorical_cols), pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out())],
    axis=1,
)

# Normalisasi fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded.drop(columns=['aveOralM']))
y = df_encoded['aveOralM'].values

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konversi ke tensor untuk PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Model Deep Learning dengan PyTorch
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

input_dim = X_train.shape[1]
model_pytorch = RegressionModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_pytorch.parameters(), lr=0.001)

# Training model PyTorch
num_epochs = 100
for epoch in range(num_epochs):
    model_pytorch.train()
    optimizer.zero_grad()
    outputs = model_pytorch(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluasi model PyTorch
model_pytorch.eval()
with torch.no_grad():
    y_pred_pytorch = model_pytorch(X_test_tensor).numpy()

mse_pytorch = mean_squared_error(y_test, y_pred_pytorch)
rmse_pytorch = sqrt(mse_pytorch)
r2_pytorch = r2_score(y_test, y_pred_pytorch)
print(f"PyTorch - MSE: {mse_pytorch:.4f}, RMSE: {rmse_pytorch:.4f}, R^2: {r2_pytorch:.4f}")

# Model Deep Learning dengan TensorFlow
model_tf = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

model_tf.compile(optimizer='adam', loss='mse')
model_tf.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluasi model TensorFlow
y_pred_tf = model_tf.predict(X_test)
mse_tf = mean_squared_error(y_test, y_pred_tf)
rmse_tf = sqrt(mse_tf)
r2_tf = r2_score(y_test, y_pred_tf)
print("TensorFlow Model Evaluation:")
print(f"MSE: {mse_tf:.4f}")
print(f"RMSE: {rmse_tf:.4f}")
print(f"R^2: {r2_tf:.4f}")

Epoch [10/100], Loss: 1356.1057
Epoch [20/100], Loss: 1324.3918
Epoch [30/100], Loss: 1273.7501
Epoch [40/100], Loss: 1191.8890
Epoch [50/100], Loss: 1070.4347
Epoch [60/100], Loss: 914.8999
Epoch [70/100], Loss: 752.2224
Epoch [80/100], Loss: 623.8290
Epoch [90/100], Loss: 542.4771
Epoch [100/100], Loss: 491.1483
PyTorch - MSE: 464.7264, RMSE: 21.5575, R^2: -2205.9699
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1300.2081 - val_loss: 1085.8685
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 987.4088 - val_loss: 714.9427
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 613.1330 - val_loss: 436.0344
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 452.5804 - val_loss: 361.9280
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 367.6575 - val_loss: 294.3271
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 295.0855 - val_loss: 223.0934
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 200.6534 - val_loss: 157.2399
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 140.0818 - val_loss: 102.5694
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 83.5720 - val_loss: 68.9553
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 49.8051 - val_loss: 46.8510
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 33.0277 - val_loss: 35.2081
Epoch 12/100
26/26 ━━━━━━━━━━━━━

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
TensorFlow Model Evaluation:
MSE: 1.5669
RMSE: 1.2518
R^2: -6.4413


#Persamaan Matematika
##1. Mean Squared Error (MSE)
Mean Squared Error adalah salah satu metrik evaluasi yang paling umum digunakan dalam regresi. MSE mengukur rata-rata kesalahan kuadrat antara nilai aktual $(y_i)$ dan nilai prediksi $(\hat{y}_i)$. Secara matematis, MSE didefinisikan sebagai:

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

Di mana:
- $n$ = jumlah sampel dalam dataset
- $y_i$ = nilai aktual dari data ke-$i$
- $\hat{y}_i$ = nilai prediksi dari model untuk data ke-$i$

MSE memberikan bobot lebih besar terhadap kesalahan besar karena adanya pemangkatan kuadrat, sehingga jika ada outlier yang memiliki selisih besar antara nilai aktual dan prediksi, nilai MSE akan meningkat secara signifikan. Oleh karena itu, model yang baik akan memiliki nilai MSE yang lebih kecil, menunjukkan bahwa prediksinya mendekati nilai sebenarnya. <br><br>

##2. Root Mean Squared Error (RMSE)
Root Mean Squared Error (RMSE) adalah akar kuadrat dari MSE dan didefinisikan sebagai:

$RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$

Di mana:
- $n$ = jumlah sampel dalam dataset
- $y_i$ = nilai aktual pada sampel ke-$i$
- $\hat{y}_i$ = nilai prediksi dari model pada sampel ke-$i$

Karena MSE menghasilkan nilai dalam skala kuadrat dari target, RMSE digunakan untuk mengembalikan skala kesalahan ke unit asli dari variabel target, sehingga lebih mudah untuk diinterpretasikan. RMSE sering digunakan dalam aplikasi praktis karena memberikan indikasi langsung tentang sejauh mana prediksi model melenceng dari nilai sebenarnya dalam satuan yang sama dengan target.

Perbedaan utama antara RMSE dan MSE adalah bahwa RMSE memiliki sensitivitas yang lebih rendah terhadap outlier dibandingkan MSE, karena nilai akhirnya dikembalikan ke skala asli setelah diakarkan. Namun, outlier tetap memiliki dampak yang besar karena efek kuadrat pada kesalahan individu sebelum diakarkan.

##3. Koefisien Determinasi ($R^2$)
Koefisien determinasi, atau $R^2$, adalah metrik yang mengukur seberapa baik model regresi dapat menjelaskan variabilitas data target. Rumusnya adalah:

$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

Di mana:
- $\bar{y}$ = rata-rata dari semua nilai aktual $y_i$.
- Penyebut dalam persamaan ini adalah variansi total dari data, yang mewakili seberapa besar variasi dalam data asli tanpa model.
- Pembilang adalah jumlah kuadrat kesalahan model, yang menunjukkan seberapa jauh prediksi dari nilai sebenarnya.

Nilai $R^2$ berada dalam rentang antara 0 hingga 1. Semakin mendekati 1, semakin baik model dalam menjelaskan variabilitas data. Jika $R^2$=1, maka model menjelaskan 100% variabilitas dalam data, artinya semua prediksi tepat. Jika $R^2$=0, berarti model tidak lebih baik daripada sekadar menggunakan rata-rata $\bar{y}$ sebagai prediksi. Dalam beberapa kasus, $R^2$ bisa bernilai negatif jika model sangat buruk dan prediksinya lebih buruk dibandingkan dengan sekadar menggunakan nilai rata-rata sebagai estimasi.